In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    master('local').\
    appName('nlp-project').\
    getOrCreate()

In [2]:
df = spark.read.csv(
    path='D:/learn-ab/learning-PySpark/sample-data/sms-spam-collection.tsv',
    inferSchema=True,
    sep='\t'
)
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [3]:
df = df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
df.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [4]:
from pyspark.sql.functions import length

In [5]:
df = df.withColumn('length',length(df['text']))
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [6]:
df.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [7]:
from pyspark.ml.feature import (
    Tokenizer,
    StopWordsRemover,
    CountVectorizer,
    IDF,
    StringIndexer
)

In [8]:
tokenizer = Tokenizer(
    inputCol='text',
    outputCol='token_text'
)

In [9]:
stop_words_remover = StopWordsRemover(
    inputCol='token_text',
    outputCol='stop_tokens'
)

In [10]:
count_vec = CountVectorizer(
    inputCol='stop_tokens',
    outputCol='count_vec'
)

In [11]:
idf = IDF(
    inputCol='count_vec',
    outputCol='tf_idf'
)

In [12]:
ham_spam_to_num = StringIndexer(
    inputCol='class',
    outputCol='label'
)

In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [14]:
assembler = VectorAssembler(
    inputCols=[
        'tf_idf',
        'length'
    ],
    outputCol='features'
)

In [15]:
from pyspark.ml.classification import NaiveBayes

In [16]:
naive_bayes = NaiveBayes()

In [17]:
from pyspark.ml import Pipeline

In [18]:
data_prep_pipe = Pipeline(
    stages=[
        ham_spam_to_num,
        tokenizer,
        stop_words_remover,
        count_vec,
        idf,
        assembler
    ]
)

In [19]:
data_prep_pipeline = data_prep_pipe.fit(df)

In [20]:
df_clean = data_prep_pipeline.transform(df)
df_clean.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|           count_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|(13424,[0,24,301,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [21]:
df_clean = df_clean.select('label', 'features')
df_clean.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [22]:
train_data, test_data = df_clean.randomSplit([0.7, 0.3])

In [23]:
spam_detector = naive_bayes.fit(train_data)
spam_results = spam_detector.transform(test_data)

In [24]:
spam_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,4,50,...|[-823.94824804744...|[1.0,4.5787781995...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-885.37071029385...|[1.0,5.7886299880...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-870.71661736307...|[1.0,1.5179443996...|       0.0|
|  0.0|(13424,[0,1,150,1...|[-250.07070457697...|[0.99996410738653...|       0.0|
|  0.0|(13424,[0,1,881,1...|[-94.252511420495...|[0.99999999714904...|       0.0|
|  0.0|(13424,[0,1,4305,...|[-127.83132072188...|[0.99998782514690...|       0.0|
|  0.0|(13424,[0,2,3,6,9...|[-3282.6953824706...|[1.0,8.2338818920...|       0.0|
|  0.0|(13424,[0,2,4,8,1...|[-1316.6433121299...|[1.0,1.9876044985...|       0.0|
|  0.0|(13424,[0,2,4,128...|[-645.50214154147...|[1.0,4.8157569940...|       0.0|
|  0.0|(13424,[0

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
spam_eval = MulticlassClassificationEvaluator(
    predictionCol='prediction',
    labelCol='label',
    metricName='f1'
)

In [27]:
spam_detection_f1_score = spam_eval.evaluate(spam_results)

In [28]:
print(f'F1 Score of model at predicting spam was: {spam_detection_f1_score:.2f}')

F1 Score of model at predicting spam was: 0.93
